In [116]:
import folium
import geopandas as gpd
import pandas as pd
import numpy as np

### View satellite image of field

In [117]:
# Spanos field at Cal Poly SLO
location = [35.298387792355854, -120.66508935519974]

m = folium.Map(location=location, zoom_start=20, max_zoom=20)

folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Esri Satellite",
    overlay=False,
    control=True,
    max_zoom=20,
).add_to(m)

folium.LayerControl().add_to(m)

m

### Plot polygon of field outline

In [118]:
# grab 4 field corners on Google Earth
ne = [35.298971, -120.664876]
se = [35.298036, -120.664575]
sw = [35.297895, -120.665234]
nw = [35.298829, -120.665535]

In [119]:
folium.Marker(nw, icon=folium.Icon(icon="d", prefix="fa", color="black")).add_to(m)
folium.Marker(ne, icon=folium.Icon(icon="c", prefix="fa", color="black")).add_to(m)
folium.Marker(sw, icon=folium.Icon(icon="b", prefix="fa", color="black")).add_to(m)
folium.Marker(se, icon=folium.Icon(icon="a", prefix="fa", color="black")).add_to(m)

folium.Polygon(
    locations=[ne, se, sw, nw],
    color="blue",
    weight=2,
    fill=True,
    fill_color="blue",
    fill_opacity=0.2,
).add_to(m)

m

### Generate data collection points

In [120]:
field_length_ft = 352
field_width_ft = 204

row_spacing_ft = 22
col_spacing_ft = 38

start_offset_x_ft = 3
start_offset_y_ft = 5

In [124]:
def interpolate_points(start, end, num_points):
    return [start + t * (end - start) for t in np.linspace(0, 1, num_points)]


# field edges
north_vector = np.array(ne) - np.array(nw)
south_vector = np.array(se) - np.array(sw)
west_vector = np.array(sw) - np.array(nw)

# offset from edge
x_fraction = start_offset_x_ft / field_width_ft
y_fraction = start_offset_y_ft / field_length_ft

# points for each column
start_point = np.array(nw) + y_fraction * north_vector + x_fraction * west_vector
num_points_per_row = 10
num_columns = 10
row_spacing_fraction = row_spacing_ft / field_width_ft
col_spacing_fraction = col_spacing_ft / field_length_ft

points_list = []

# start at 100 and count down, based on how we collected the data
point_id = 100
for col in range(num_columns):
    col_start_point = start_point + col * col_spacing_fraction * north_vector
    column_points = interpolate_points(
        col_start_point,
        col_start_point + row_spacing_fraction * west_vector * (num_points_per_row - 1),
        num_points_per_row,
    )
    for point in column_points:
        points_list.append(
            {"id": point_id, "latitude": point[0], "longitude": point[1]}
        )
        point_id -= 1

points_data = pd.DataFrame(points_list)
points_data.head()

,id,latitude,longitude
0,100,35.298817,-120.665521
1,99,35.298717,-120.665489
2,98,35.298616,-120.665456
3,97,35.298515,-120.665424
4,96,35.298414,-120.665391


In [125]:
for idx, row in points_data.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5,
        color=None,
        fill=True,
        fill_color="red",
        fill_opacity=1,
        popup=f'Point {int(row["id"])}',
    ).add_to(m)
m

### Save csv of georeferenced points

In [126]:
points_data.to_csv("./points.csv", index=False)